### SImple Genai app using Langchain.

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

api_key2 = os.getenv("OPENAI_API_KEY")
api_langchain = os.getenv("LANGCHAIN_API_KEY")

# ✅ Ensure 'LANGCHAIN_PROJECT' is not None before setting it
langchain_project = os.getenv("LANGCHAIN_PROJECT", "default_project")  # 👈 Provide a default value

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = langchain_project

In [4]:
##Data ingestion -- from the website
from langchain_community.document_loaders import WebBaseLoader


In [5]:
loader = WebBaseLoader("https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html")
loader

In [6]:
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nChatPromptTemplate — 🦜🔗 LangChain  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\nSection Navigation\nBase packages\n\nCore\nagents\nbeta\ncaches\ncallbacks\nchat_history\nchat_loaders\nchat_sessions\ndocument_loaders\ndocuments\nembeddings\nexample_selectors\nexceptions\nglobals\nindexing\nlanguage_models\nload\nmessages\nout

In [10]:
### Load data -->then divide to chunks--->embedding to vector from text-->then store in vector embedding
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)


In [11]:
documents

[Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_content='ChatPromptTemplate — 🦜🔗 LangChain  documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nBack to top\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\n\nCtrl+K\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    Reference\n  \n\n\n\n\n\n\n\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\nGitHub\n\n\n\nX / Twitter\n\n\n\n\n\n\n\nSection Navigation\nBase packages'),
 Document(metadata={'source': 'https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html', 'title': 'ChatPromptTemplate — 🦜🔗 LangChain  documentation', 'language': 'en'}, page_c

In [14]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=api_key2,model="text-embedding-3-large")

In [15]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [16]:
vectorstoredb

In [17]:
##direct query to vector db 
query = "Below is a table that illustrates some events that might be emitted by various chains. Metadata fields have been omitted from the table for brevity. Chain definitions have been included after the table."
result = vectorstoredb.similarity_search(query)

In [19]:
print(result[0].page_content)

tags: Optional[List[str]] - The tags of the Runnable that generatedthe event.




metadata: Optional[Dict[str, Any]] - The metadata of the Runnablethat generated the event.



data: Dict[str, Any]

Below is a table that illustrates some events that might be emitted by various
chains. Metadata fields have been omitted from the table for brevity.
Chain definitions have been included after the table.
ATTENTION This reference table is for the V2 version of the schema.


event
name
chunk
input
output



on_chat_model_start
[model name]

{“messages”: [[SystemMessage, HumanMessage]]}


on_chat_model_stream
[model name]
AIMessageChunk(content=”hello”)



on_chat_model_end
[model name]

{“messages”: [[SystemMessage, HumanMessage]]}
AIMessageChunk(content=”hello world”)

on_llm_start
[model name]

{‘input’: ‘hello’}


on_llm_stream
[model name]
‘Hello’



on_llm_end
[model name]

‘Hello human!’


on_chain_start
format_docs




on_chain_stream
format_docs
“hello world!, goodbye world!”


In [53]:
from langchain_openai import ChatOpenAI
llm  = ChatOpenAI(api_key=api_key2,model="gpt-4-turbo-preview")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x12671b6a0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x126761000> root_client=<openai.OpenAI object at 0x1266da2c0> root_async_client=<openai.AsyncOpenAI object at 0x12671b940> model_name='gpt-4-turbo-preview' model_kwargs={} openai_api_key=SecretStr('**********')


In [60]:
### Retrievel Chain , document chain

from langchain.chains.combine_documents import create_stuff_documents_chain #create a chain to pass a list of document to a model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that answers questions based on the provided context."),
    ("human", "{question}"),
    ("assistant", "I'll help answer your question based on the context provided."),
    MessagesPlaceholder(variable_name="context"),
])

document_chain = create_stuff_documents_chain(llm,prompt)

document_chain 

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={'context': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.Ch

### We can use retriever to dynamically select the most relevent documents and pass those in for a given question.

In [64]:
###input ---> Retriver ---->vectordb
retriever = vectorstoredb.as_retriever()
retrieval_chain = (
    retriever
    | document_chain
)
from langchain.chains import create_retrieval_chain
 ##combine retiver and document_chain , retirver for to get relevent data from vectordbs and docs_chain for context


In [74]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | document_chain
)

In [75]:
# Example usage
query = "What is LangChain?"
response = rag_chain.invoke(query)
print(response)

ValueError: variable context should be a list of base messages, got How to chain runnables
How to convert Runnables as Tools
How to create a custom LLM class
How to create a dynamic (self-constructing) chain
How to create custom callback handlers
How to create tools
How to deal with large databases when doing SQL question-answering
How to debug your LLM apps
How to do per-user retrieval
How to do query validation as part of SQL question-answering
How to do question answering over CSVs
How to do tool/function calling
How to get a RAG application to add citations
How to get your RAG application to return sources
How to handle cases where no queries are generated
How to handle long text when doing extraction
How to handle multiple queries when doing query analysis
How to handle multiple retrievers when doing query analysis
How to handle tool errors
How to inspect runnables
How to invoke runnables in parallel
How to map values to a graph database
How to migrate from legacy LangChain agents